# Best Strategy in Finding a Business Location

To find the best business location, start by analyzing your target market, researching potential areas, and considering factors like accessibility, infrastructure, and zoning regulations.
Here's a more detailed strategy:  

##  1. Define Your Needs and Goals: 
- **Target Market:** Identify your ideal customer base and where they are located
    - For definition see [this](https://www.investopedia.com/terms/t/target-market.asp#:~:text=Demographic%3A%20These%20are%20the%20main,in%20the%20era%20of%20globalization.)
- **Business Type:** Determine if your business requires 
    - high foot traffic
    - proximity to suppliers
    - access to specific infrastructure (e.g., highways, rail yards)
- **Budget:** Establish a realistic budget for 
    - rent
    - utilities
    - other location-related costs
- **Brand Image:** Consider how the location will impact your brand's image and customer perception

### Market Analysis
Example: Laundromat in Vandergrift, PA

- **Demographic Analysis:** Use local census data to understand the demographics of your area. 
    - Look for age distributions, income levels, and household sizes that match the typical profile of a user (for our example laundromat user).
- **Psychographic Information:** Go beyond basic demographics to explore customer lifestyles, values, and habits. 

For instance, are there large groups of environmentally conscious consumers in your area who might appreciate eco-friendly laundry solutions?

In [ ]:
# Import modules
import matplotlib.pyplot as plt
import pandas as pd
from census import Census
from us import states

# Created modules
from ipython_config import CENSUS_KEY


In [ ]:
# Set API key
c = Census(CENSUS_KEY)

In [ ]:
# Coin-Operated Laundries and Drycleaners
NAICS = '812310'
# Vandergrift, PA
ZIPCODE = '15690'
STATEFIPS = states.PA.fips
NAME = 'Pennsylvania'

### Poverty Rate by State, County, Tract

In [ ]:
# Obtain Census variables from the 2019 ACS at the tract level for the Commonwealth of Virginia (FIPS code: 51)
# C17002_001E: count of ratio of income to poverty in the past 12 months (total)
# C17002_002E: count of ratio of income to poverty in the past 12 months (< 0.50)
# C17002_003E: count of ratio of income to poverty in the past 12 months (0.50 - 0.99)
# B01003_001E: total population
# Sources: https://api.census.gov/data/2019/acs/acs5/variables.html; https://pypi.org/project/census/
census = c.acs5.state_county_tract(fields = ('NAME', 'C17002_001E', 'C17002_002E', 'C17002_003E', 'B01003_001E'),
                                      state_fips = STATEFIPS,
                                      county_fips = "*",
                                      tract = "*",
                                      year = 2022)

In [ ]:
# Create a dataframe from the census data
df = pd.DataFrame(census)

# Show the dataframe
print(df.head(2))
print('Shape: ', df.shape)

In [ ]:
# Combine state, county, and tract columns together to create a new string and assign to new column
df["GEOID"] = df["state"] + df["county"] + df["tract"]

In [ ]:
# Remove columns
df = df.drop(columns = ["state", "county", "tract"])

# Show updated dataframe
df.head(2)

In [ ]:
# Check column data types for census data
print("Column data types for census data:\n{}".format(df.dtypes))

# Source: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dtypes.html

In [ ]:
# Get poverty rate and store values in new column
df["Poverty_Rate"] = (df["C17002_002E"] + df["C17002_003E"]) / df["B01003_001E"] * 100

# Show dataframe
df[['NAME', 'Poverty_Rate']].head()

## Annual Business Survey (ABS) Data

In [ ]:
# Company Summary

abscs_params = {
    'variables' : """GEO_ID,NAME,NAICS2022,NAICS2022_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,VET_GROUP,VET_GROUP_LABEL,EMPSZFI,EMPSZFI_LABEL,YEAR,FIRMPDEMP,FIRMPDEMP_F,RCPPDEMP,RCPPDEMP_F,EMP,EMP_F,PAYANN,PAYANN_F,FIRMPDEMP_S,FIRMPDEMP_S_F,RCPPDEMP_S,RCPPDEMP_S_F,EMP_S,EMP_S_F,PAYANN_S,PAYANN_S_F""",
    'geography': f"state:{STATEFIPS}",
    'api_key': CENSUS_KEY,
    'dataset_base': 'abscs',
    'year': '2022'
}

In [ ]:
# Characteristics of Businesses

abscb_params = {
    'variables' : """GEO_ID,NAME,NAICS2022,NAICS2022_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,VET_GROUP,VET_GROUP_LABEL,QDESC,QDESC_LABEL,BUSCHAR,BUSCHAR_LABEL,YEAR,FIRMPDEMP,FIRMPDEMP_F,FIRMPDEMP_PCT,FIRMPDEMP_PCT_F,RCPPDEMP,RCPPDEMP_F,RCPPDEMP_PCT,RCPPDEMP_PCT_F,EMP,EMP_F,EMP_PCT,EMP_PCT_F,PAYANN,PAYANN_F,PAYANN_PCT,PAYANN_PCT_F,FIRMPDEMP_S,FIRMPDEMP_S_F,FIRMPDEMP_PCT_S,FIRMPDEMP_PCT_S_F,RCPPDEMP_S,RCPPDEMP_S_F,RCPPDEMP_PCT_S,RCPPDEMP_PCT_S_F,EMP_S,EMP_S_F,EMP_PCT_S,EMP_PCT_S_F,PAYANN_S,PAYANN_S_F,PAYANN_PCT_S,PAYANN_PCT_S_F""",
    'geography': "us:*",
    'api_key': CENSUS_KEY,
    'dataset_base': 'abscb',
    'year': '2022'
}


In [ ]:

# Characteristics of Business Owners
abscbo_params = {
    'variables' : """GEO_ID,NAME,NAICS2022,NAICS2022_LABEL,OWNER_SEX,OWNER_SEX_LABEL,OWNER_ETH,OWNER_ETH_LABEL,OWNER_RACE,OWNER_RACE_LABEL,OWNER_VET,OWNER_VET_LABEL,QDESC,QDESC_LABEL,OWNCHAR,OWNCHAR_LABEL,YEAR,OWNPDEMP,OWNPDEMP_F,OWNPDEMP_PCT,OWNPDEMP_PCT_F,OWNPDEMP_S,OWNPDEMP_S_F,OWNPDEMP_PCT_S,OWNPDEMP_PCT_S_F""",
    'geography': "us:*",
    'api_key': CENSUS_KEY,
    'dataset_base': 'abscbo',
    'year': '2022'
}

In [ ]:
# Module Business Characteristics
absmcb_params = {
    'variables' : """GEO_ID,NAME,NAICS2022,NAICS2022_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,VET_GROUP,VET_GROUP_LABEL,QDESC,QDESC_LABEL,BUSCHAR,BUSCHAR_LABEL,YEAR,FIRMPDEMP,FIRMPDEMP_F,FIRMPDEMP_PCT,FIRMPDEMP_PCT_F,RCPPDEMP,RCPPDEMP_F,RCPPDEMP_PCT,RCPPDEMP_PCT_F,EMP,EMP_F,EMP_PCT,EMP_PCT_F,PAYANN,PAYANN_F,PAYANN_PCT,PAYANN_PCT_F,FIRMPDEMP_S,FIRMPDEMP_S_F,FIRMPDEMP_PCT_S,FIRMPDEMP_PCT_S_F,RCPPDEMP_S,RCPPDEMP_S_F,RCPPDEMP_PCT_S,RCPPDEMP_PCT_S_F,EMP_S,EMP_S_F,EMP_PCT_S,EMP_PCT_S_F,PAYANN_S,PAYANN_S_F,PAYANN_PCT_S,PAYANN_PCT_S_F""",
    'geography': "us:*",
    'api_key': CENSUS_KEY,
    'dataset_base': 'absmcb',
    'year': '2022'
}

In [ ]:
import requests

def get_abs_data(abs_params):
    """
    Get data from the Annual Business Survey (ABS) API
    
    Parameters:
    - variables: Comma-separated string of variable names
    - geography: Geographic level (default: national level)
    - api_key: Your Census Bureau API key
    """
    base_url = "https://api.census.gov/data"
    year = abs_params['year']
    dataset = abs_params['dataset_base']
    variables = abs_params['variables']
    geography = abs_params['geography']
    api_key = abs_params['api_key']
    
    # Construct the full URL
    url = f"{base_url}/{year}/{dataset}?get={variables}&for={geography}"
    
    if api_key:
        url += f"&key={api_key}"
        
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Failed to retrieve data. Status code: {response}")

# Example usage
try:
    # Get basic business statistics nationally
    data = get_abs_data(abscs_params)
except Exception as e:
    print(f"Error: {str(e)}")

In [ ]:
usb = pd.DataFrame(data)
usb.head()

In [ ]:
usb.info()

In [ ]:
usb[2].value_counts()

In [ ]:
usb[(usb[2]==NAICS)].head()

## 2. Research Potential Locations:
- **Demographics:** Research the demographics of potential areas to ensure they align with your target market
- **Competition:** Analyze the competitive landscape in each area to understand the level of competition
- **Infrastructure:** Assess the availability of essential infrastructure, such as 
    - transportation
    - utilities
    - internet access
- **Zoning Laws:** Understand local zoning regulations and restrictions to ensure compliance
- **Local Resources:** Explore local government census data or tools like WIGeoLocation for further research  
- **Accessibility:** Evaluate the accessibility of the location for both customers and employees
- **Traffic:** Consider traffic patterns and parking availability, especially if foot traffic is important 

## 3. Evaluate and Compare:  
- **Visit Potential Locations:** Conduct site visits to assess the suitability of each location firsthand
- **Cost Analysis:** Compare the costs of different 
    - locations
    - including rent
    - utilities
    - taxes  
- **Pros and Cons:** Create a list of pros and cons for each potential location to help make a decision
- **Long-Term Growth Potential:** Consider the long-term growth potential of the area and its ability to support your business  

## 4. Consider Legal and Institutional Factors: 
- **Business Licenses:** Research the necessary business licenses and permits required for your business type and location. 
- **Local Regulations:** Understand any local regulations or restrictions that may affect your business operations.
- **Government Incentives:** Explore any local or state government incentives or programs that may be available for businesses in specific areas.